In [1]:
#Dudas: el merge no lo llegué a entender del todo junto con la creación del index 
# en desanidar.py

#borrar credits.csv y movies csv antes de enviar

In [2]:
#Importamos librerias
import pandas as pd
import sys
import os

In [3]:
#Agregamos la ruta de la carpeta functions
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))

                            **Extracción y Transformación Cast**                             

In [4]:
#importamos el convertidor de archivos a parquet y creamos el Dataframe con dicha conversión:
from csv_a_parquet import convertidor_parquet
credits_df = convertidor_parquet('../data/credits.csv', '../data/credits.parquet')

In [5]:
#Confirmamos la carga:
credits_df.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


                                    **Desanidar Cast**                                  

In [6]:
#Observamos que el dataframe está compuesto por str
credits_df['cast'].apply(type).value_counts()

cast
<class 'str'>    45476
Name: count, dtype: int64

In [7]:
#Importamos la función para generar la conversión:
from convertidor_list_dict import convertir_list_dict
credits_df['cast'] = credits_df['cast'].apply(convertir_list_dict)

In [8]:
#Confirmamos el cambio:
credits_df['cast'].apply(type).value_counts() 

cast
<class 'list'>    45476
Name: count, dtype: int64

In [9]:
# Importamos desanidar_columna para proceder a desanidar la columna cast:
from desanidar import desanidar_columna
cast = desanidar_columna(credits_df, 'cast', 'cast')

# Confirmamos la creación de la nueva tabla:
cast.head(2)

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg


In [10]:
# Hacemos un merge para añadir los id de la tabla credits_id con el objetivo de 
# Relacionar la tabla cast con movies:

cast = cast.merge(credits_df[['cast_id', 'id']], on='cast_id', 
                                        how='left', suffixes=('', '_credits'))

In [12]:
# Dropeamos las filas que NaN en casi todas lasd columnas:
cast.dropna(subset=[col for col in cast.columns 
                    if col not in ['cast_id', 'id_credits']], how='all', inplace=True)

In [13]:
# Eliminamos las columnas 'order' y 'profile_path' ya que no son utiles para le analisis
cast.drop(labels=['order', 'profile_path'], axis=1, inplace=True)

# Eliminamos las columnas 'character' y 'gender' ya que no serán utiles para el analisis
cast.drop(labels=['character', 'gender'], axis=1, inplace=True)

# Transformamos la columna name y id para no crear conflictos con al API
cast['name'] = cast['name'].str.lower()
cast['id'] = cast['id'].astype(int)


In [14]:
cast.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562474 entries, 0 to 564890
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   cast_id     562474 non-null  int64 
 1   credit_id   562474 non-null  object
 2   id          562474 non-null  int64 
 3   name        562474 non-null  object
 4   id_credits  562474 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 25.7+ MB


                                    **Carga Cast**                                           

In [ ]:
credits = '../data_post_etl/credits/'
os.makedirs(credits, exist_ok=True)
ruta = os.path.join(credits, 'cast_transformados.parquet')
cast.to_parquet(ruta, index=False)

                            **Transformación Crew**                                 

In [16]:
#Observamos que el dataframe está compuesto por str
credits_df['crew'].apply(type).value_counts()

crew
<class 'str'>    45476
Name: count, dtype: int64

In [17]:
#Importamos la función para generar la conversión y desanidar:
credits_df['crew'] = credits_df['crew'].apply(convertir_list_dict)

In [18]:
#Confirmamos el cambio:
credits_df['cast'].apply(type).value_counts()

cast
<class 'list'>    45476
Name: count, dtype: int64

In [19]:
# Importamos desanidar_columna para proceder a desanidar la columna crew:
crew = desanidar_columna(credits_df, 'crew', 'crew', 1)

# Confirmamos la creación de la nueva tabla:
crew.head(3)

,credit_id,department,gender,id,job,name,profile_path,crew_id
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,1
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,1
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,1


In [20]:
# Hacemos un merge para añadir los id de la tabla credits_id con el objetivo de 
# Relacionar la tabla crew con movies:
crew = crew.merge(credits_df[['crew_id', 'id']], on='crew_id', 
                                        how='left', suffixes=('', '_credits'))

In [21]:
# Dropeamos las filas que NaN en casi todas lasd columnas:
crew.dropna(subset=[col for col in crew.columns 
                    if col not in ['crew_id', 'id_credits']], how='all', inplace=True)
                    

In [22]:
crew.head(1)

,credit_id,department,gender,id,job,name,profile_path,crew_id,id_credits
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,1,862


In [23]:
# Del dataset Crew solo nos interesa el director, por lo tanto:
crew = crew[crew['job'] == 'Director']

# Eliminamos las columnas 'gender' 'departament' y 'profile_path' ya que no son 
# utiles para el analisis
crew.drop(labels=['gender', 'profile_path', 'department'], axis=1, inplace=True)

# Transformamos la columna name y id para no crear conflictos con al API
crew['name'] = crew['name'].str.lower()
crew['id'] = crew['id'].astype(int)

                                    **Carga Crew**                                   

In [24]:
credits = '../data_post_etl/credits/'
os.makedirs(credits, exist_ok=True)
ruta = os.path.join(credits, 'crew_transformados.parquet')
crew.to_parquet(ruta, index=False)

In [25]:
""" Ver si hay que agregar esto al final: 
from convert_list_to_str import lista_a_str
# Aplicar la función de conversión a la columna
df['cast'] = df['cast'].apply(lista_a_str)
df['crew'] = df['crew'].apply(lista_a_str)
# Asegúrate de que el directorio de destino exista
output_dir = '../data/processed/credits/'
os.makedirs(output_dir, exist_ok=True)"""

" Ver si hay que agregar esto al final: \nfrom convert_list_to_str import lista_a_str\n# Aplicar la función de conversión a la columna\ndf['cast'] = df['cast'].apply(lista_a_str)\ndf['crew'] = df['crew'].apply(lista_a_str)\n# Asegúrate de que el directorio de destino exista\noutput_dir = '../data/processed/credits/'\nos.makedirs(output_dir, exist_ok=True)"